# 分类数据过程

In [1]:
# 导入分类数据（5mer）
import pandas as pd
import numpy as np
pos_5mer = pd.read_csv('/home/xlxiang/biostatistics/data/Classification/pos_5mer.csv',header=None)
neg_5mer = pd.read_csv('/home/xlxiang/biostatistics/data/Classification/neg_5mer.csv',header=None)
all_data = pd.concat([pos_5mer,neg_5mer],axis=0)
all_data[1024]=np.concatenate((np.ones(6606,dtype=np.int64),np.zeros(6606,dtype=np.int64)))

In [2]:
#划分为训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data.iloc[:,:1024], all_data.iloc[:,1024], random_state=2023)
#计算训练集和测试集的acc、precision、recall、f1 score、auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
def calculate_result(model,x_train, y_train,x_test, y_test):
    train_predict = model.predict(x_train)
    test_predict = model.predict(x_test)
    train_pro = model.predict_proba(x_train)
    test_pro = model.predict_proba(x_test)

    train_acc = accuracy_score(y_train, train_predict)
    train_precision = precision_score(y_train, train_predict)
    train_recall = recall_score(y_train, train_predict)
    train_f1 = f1_score(y_train, train_predict)
    train_auc = roc_auc_score(y_train, train_pro[:,1])

    test_acc = accuracy_score(y_test, test_predict)
    test_precision = precision_score(y_test, test_predict)
    test_recall = recall_score(y_test, test_predict)
    test_f1 = f1_score(y_test, test_predict)
    test_auc = roc_auc_score(y_test, test_pro[:,1])

    print("train_acc: {0:.3f}, test_acc: {1:.3f}".format(train_acc, test_acc))
    print("train_pre: {0:.3f}, test_pre: {1:.3f}".format(train_precision, test_precision))
    print("train_recall: {0:.3f}, test_recall: {1:.3f}".format(train_recall, test_recall))
    print("train_f1: {0:.3f}, test_f1: {1:.3f}".format(train_f1, test_f1))
    print("train_auc: {0:.3f}, test_auc: {1:.3f}".format(train_auc, test_auc))
    return None

## logistic regression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#Logistic寻参
params = {'penalty': ['l1', 'l2'],
          'C': [10, 50, 100, 150, 200, 500, 600, 800, 1000]}
model_logistic = GridSearchCV(LogisticRegression(), param_grid=params, cv=10, n_jobs = -1)
model_logistic.fit(X_train, y_train)
print("Best parameters: {}".format(model_logistic.best_params_))
print("Best cross-validation score: {:.2f}".format(model_logistic.best_score_))
calculate_result(model_logistic,X_train,y_train,X_test,y_test)

/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/skle

Best parameters: {'C': 800, 'penalty': 'l2'}
Best cross-validation score: 0.91


/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/skle

train_acc: 0.929, test_acc: 0.906
train_pre: 0.927, test_pre: 0.907
train_recall: 0.931, test_recall: 0.907
train_f1: 0.929, test_f1: 0.907
train_auc: 0.981, test_auc: 0.968


/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#KNN寻参
params = {'p': [1,2], 'n_neighbors': [i for i in range(3, 25, 2)]}
model_knn = GridSearchCV(KNeighborsClassifier(), param_grid=params, cv=10, n_jobs = -1)
model_knn.fit(X_train, y_train)
print("Best parameters: {}".format(model_knn.best_params_))
print("Best cross-validation score: {:.2f}".format(model_knn.best_score_))
calculate_result(model_knn,X_train,y_train,X_test,y_test)

Best parameters: {'n_neighbors': 17, 'p': 2}
Best cross-validation score: 0.90
train_acc: 0.916, test_acc: 0.897
train_pre: 0.942, test_pre: 0.927
train_recall: 0.886, test_recall: 0.863
train_f1: 0.914, test_f1: 0.894
train_auc: 0.978, test_auc: 0.965


## LDA

In [12]:
from sklearn import discriminant_analysis
#LDA
lda = discriminant_analysis.LinearDiscriminantAnalysis()
calculate_result(lda,X_train,y_train,X_test,y_test)

train_acc: 0.932, test_acc: 0.896
train_pre: 0.929, test_pre: 0.898
train_recall: 0.935, test_recall: 0.895
train_f1: 0.932, test_f1: 0.896
train_auc: 0.982, test_auc: 0.962


## SVM

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#SVM寻参
params = [
    {'kernel': ['linear'], 'C': [0.01,0.1,1,10]},
    {'kernel': ['rbf'], 'C': [10,100,500], 'gamma': [0.01,0.1,1,10]},
]
model_svm = GridSearchCV(SVC(probability=True), param_grid=params, cv=10, n_jobs = 5)
model_svm.fit(X_train, y_train)
print("Best parameters: {}".format(model_svm.best_params_))
print("Best cross-validation score: {:.2f}".format(model_svm.best_score_))
calculate_result(model_svm,X_train,y_train,X_test,y_test)

Best parameters: {'C': 10, 'gamma': 10, 'kernel': 'rbf'}
Best cross-validation score: 0.92
train_acc: 0.933, test_acc: 0.909
train_pre: 0.926, test_pre: 0.905
train_recall: 0.940, test_recall: 0.916
train_f1: 0.933, test_f1: 0.910
train_auc: 0.981, test_auc: 0.969


## RF

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#RF寻参
params = {'n_estimators':range(300,1000,100),
    'max_depth':range(6,43,6),
    'max_features': range(30,90,5)}
model_rf = GridSearchCV(RandomForestClassifier(),param_grid=params, cv=10, n_jobs=5)
model_rf.fit(X_train, y_train)
print("Best parameters: {}".format(model_rf.best_params_))
print("Best cross-validation score: {:.2f}".format(model_rf.best_score_))
calculate_result(model_rf,X_train,y_train,X_test,y_test)

Best parameters: {'max_depth': 30, 'max_features': 65, 'n_estimators': 600}
Best cross-validation score: 0.90
train_acc: 1.000, test_acc: 0.900
train_pre: 1.000, test_pre: 0.882
train_recall: 1.000, test_recall: 0.925
train_f1: 1.000, test_f1: 0.903
train_auc: 1.000, test_auc: 0.964
